In [ ]:
%matplotlib inline

#
# import utilitaires
#
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random

import helpers as hlp
import helpers.dataset.PlantVillage as pv
import helpers.features as feat

In [ ]:
#
# parametres dataset PlantVillage
#
config_overrides = hlp.MetaObject.from_json("config_overrides.json")
executor = hlp.create_thread_pool_executor(max_workers=None)

pv_config = pv.Config(executor)
if not config_overrides is None:
    hlp.MetaObject.override_from_object(pv_config, 
                                        config_overrides.dataset)

print("pv_config")
display(vars(pv_config))

In [ ]:
#
# obtenir le dataset PlantVillage
#
if False:
    %prun pv_dataset = pv.load(pv_config)
else:
    pv_dataset = pv.load(pv_config)
    
if pv_dataset is None:
    print("Invalid dataset")
else:
    pv_dataframe = pv_dataset.dataframe

print("PlantVillage", pv_dataframe.shape)
display( pv_dataframe.head() )

In [ ]:
#
# parametres pour features
#
features_config = feat.OrbFeaturesConfig(executor)
if not config_overrides is None:
    hlp.MetaObject.override_from_object(features_config, 
                                        config_overrides.orb_features)

print("features_config")
display(vars(features_config))

#
# obtenir les features pour chaque image
#
orb_iter = feat.DatasetIter(pv_dataset, 
                            pv_dataframe["image_path"].iteritems(),
                            pv_dataframe.shape[0])

if "features_orb" in locals():
    del features_orb

if False:
    %prun features_orb = feat.load(features_config, orb_iter)
else:
    features_orb = feat.load(features_config, orb_iter)

print("ORB features", features_orb.features.shape)
print("ORB keypoints", features_orb.key_points.shape)

In [ ]:
index_start = random.randint(0, pv_dataframe.shape[0])
index_count = 3 * 4

indices = slice(index_start, index_start + index_count - 1)

orb_iter = feat.DatasetIter(pv_dataset, 
                            pv_dataframe.loc[indices, "image_path"].iteritems())

num_rows = int(index_count / 3 + 0.5)
_, axes = plt.subplots(num_rows, 3, figsize=(12, num_rows * 4))
for ax, \
   (image_index, \
    image_path, \
    key_points_count, \
    key_points_image) in zip(axes.flatten(), feat.key_points_iter(features_orb, orb_iter)):
    plt.sca(ax)
    
    head, tail = os.path.split(image_path)
    _, head = os.path.split(head)
    
    plt.title(head + "\n" + tail + "\n" + str(key_points_count))
    plt.imshow(key_points_image)

plt.suptitle("ORB")
plt.tight_layout()
plt.show()

In [ ]:
#
# reduire la quantity de features avec KMeans 
#

In [ ]:
# #
# # split train/test
# #
# train, test = train_test_split(pv_dataframe, 
#                                train_size=0.8, 
#                                random_state=42, 
#                                stratify=pv_dataframe["label"])

# print(train.shape)
# print(test.shape)